In [27]:
!pip install langchain-groq
!pip install langchain_community
!pip install chromadb

In [28]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [29]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/software-engineer-iii-itc/job/R-68361")
page_data = loader.load().pop().page_content
print(page_data)





















Software Engineer III, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu





In [30]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ###SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ###INSTRUCTIONS:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing
        following keys: 'role', 'experience', 'skills' and 'description'.
        Only return the valid JSON
        ### VALID JSON(NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data': page_data})
print(response.content)

```json
{
  "role": "Software Engineer III, ITC",
  "experience": "6+ years",
  "skills": [
    "Java 11+",
    "Springboot 2+",
    "Gradle 6+",
    "Junit",
    "MongoDB",
    "DynamoDB",
    "PostGre",
    "GraphQL",
    "RESTFul services",
    "Docker",
    "AWS – ECS, S3, SQS, Lambda",
    "JMeter"
  ],
  "description": "As a Senior Software Engineer, you will be a prolific code contributor and will be expected to demonstrate code and product ownership. You will be responsible for advising product owners on technology-related business problems; formulating options and assessing their relative merits and risk to determine the best solution."
}
```


In [31]:
type(response.content)

str

In [32]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
json_response

{'role': 'Software Engineer III, ITC',
 'experience': '6+ years',
 'skills': ['Java 11+',
  'Springboot 2+',
  'Gradle 6+',
  'Junit',
  'MongoDB',
  'DynamoDB',
  'PostGre',
  'GraphQL',
  'RESTFul services',
  'Docker',
  'AWS – ECS, S3, SQS, Lambda',
  'JMeter'],
 'description': 'As a Senior Software Engineer, you will be a prolific code contributor and will be expected to demonstrate code and product ownership. You will be responsible for advising product owners on technology-related business problems; formulating options and assessing their relative merits and risk to determine the best solution.'}

In [33]:
type(json_response)

dict

In [34]:
import pandas as pd
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1XaG-1pH3aui3bYeTeR5Yl7qutjcz3_6l')

In [35]:
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [36]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={'links': row["Links"]},
                       ids=[str(uuid.uuid4())])


In [37]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])

In [38]:
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/kotlin-backend

In [39]:
job = json_response
job['skills']

['Java 11+',
 'Springboot 2+',
 'Gradle 6+',
 'Junit',
 'MongoDB',
 'DynamoDB',
 'PostGre',
 'GraphQL',
 'RESTFul services',
 'Docker',
 'AWS – ECS, S3, SQS, Lambda',
 'JMeter']

In [40]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):

    """
)

chain_email = prompt_email | llm
response = chain_email.invoke(input={"job_description": str(job), "link_list": links})
print(response.content)

Subject: Expert Software Engineering Services for Your Business Needs

Dear Hiring Manager,

I came across the job description for a Software Engineer III, ITC, and I believe AtliQ, an AI & Software Consulting company, can fulfill your requirements. With over 6 years of experience in software development, our team of experts can provide tailored solutions to meet your business needs.

Our team has extensive experience in Java 11+, Springboot 2+, Gradle 6+, Junit, MongoDB, DynamoDB, PostGre, GraphQL, RESTFul services, Docker, and AWS – ECS, S3, SQS, Lambda. We have a proven track record of delivering high-quality software solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

I'd like to bring to your attention some of our notable portfolios that demonstrate our capabilities:
- Java Portfolio: https://example.com/java-portfolio
- Kotlin Backend Portfolio: https://example.com/kotlin-backend-portfolio
- Devops Portfolio: https://exampl